In [1]:
import torch

Reference material: https://jhui.github.io/2018/02/09/PyTorch-Variables-functionals-and-Autograd/

In [2]:
x = torch.ones( (2,2), requires_grad = True )
print( x.requires_grad )
print( x.data)

True
tensor([[1., 1.],
        [1., 1.]])


## Forward

In [3]:
y = x+2
z = 2*y*y
out = z.mean()

In [4]:
print( out.grad_fn )
print( out.grad_fn.next_functions )
print( out.grad_fn.next_functions[0][0].next_functions )

((<MulBackward0 object at 0x1257e1450>, 0),)
((<MulBackward0 object at 0x1257d6fd0>, 0), (<AddBackward0 object at 0x1257e1490>, 0))


## Backward
- Computes the gradient

In [5]:
print( out.backward() )

None


In [6]:
print( x.data )
print( x.grad )

tensor([[1., 1.],
        [1., 1.]])
tensor([[3., 3.],
        [3., 3.]])


In [7]:
print( y.data )
print( y.grad )

tensor([[3., 3.],
        [3., 3.]])
None


In [8]:
print( z.data )
print( z.grad )

tensor([[18., 18.],
        [18., 18.]])
None


In [9]:
print( out.data )
print( out.grad )

tensor(18.)
None


## Quick Backprop 

### Params

In [10]:
N = 64        # number of samples
H = 100       # hidden layer size
D_in = 1000   # input dimensionality 
D_out = 10    # output dimensionality 
learning_rate = 1e-6

### Initialize 
<img src="./NN-labelled.png" />

In [11]:
x  = torch.randn( (N,D_in), requires_grad=True)

w10 = torch.randn( (D_in,H), requires_grad=True)
w21 = torch.randn( (H,D_out), requires_grad=True)

y = torch.randn( (N, D_out), requires_grad=True)

Start Training

In [12]:
for epoch in range(500):
    
    y_pred = x.mm(w10).clamp(min=0).mm(w21)
    loss = (y_pred - y).pow(2).sum()
    
    if( epoch %100 == 0 ):
        print( "epoch:{} loss:{}".format(epoch,loss) )
    
    loss.backward();
    
    w10.data -= learning_rate * w10.grad.data
    w21.data -= learning_rate * w21.grad.data
    
    w10.grad.data.zero_()
    w21.grad.data.zero_()

epoch:0 loss:22917328.0
epoch:100 loss:315.5669250488281
epoch:200 loss:1.3498003482818604
epoch:300 loss:0.009448150172829628
epoch:400 loss:0.00020994539954699576


Verify the trained model

In [13]:
import random
index = random.randint(0, x.shape[0]);
X_test = x[index].reshape((1,-1))
Y_test = y[index].reshape((-1,1))

print( X_test.shape )
print( w10.shape )
print( w21.shape )
print( Y_test.shape )

torch.Size([1, 1000])
torch.Size([1000, 100])
torch.Size([100, 10])
torch.Size([10, 1])


In [14]:
y_pred = X_test.mm(w10).clamp(min=0).mm(w21)

In [15]:
print( y_pred[0].data.reshape(-1,1) )
print( Y_test.data )

tensor([[ 0.8771],
        [ 0.3966],
        [-0.3747],
        [-1.5237],
        [ 0.6698],
        [ 0.4046],
        [ 0.5886],
        [ 1.6233],
        [ 0.5813],
        [-0.4827]])
tensor([[ 0.8771],
        [ 0.3969],
        [-0.3752],
        [-1.5226],
        [ 0.6698],
        [ 0.4047],
        [ 0.5882],
        [ 1.6234],
        [ 0.5814],
        [-0.4829]])
